In [1]:
import random

import cntk as C
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder

In [2]:
num_samples = 60000
batch_size = 64
learning_rate = 0.1

In [3]:
class BatchReader(object):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.num_sample = data.shape[0]

    def next_batch(self, batch_size):
        index = random.sample(range(self.num_sample), batch_size)
        return self.data[index, :].astype(float), self.label[index, :].astype(float)

In [4]:
mnist = fetch_openml('mnist_784')

In [5]:
data = mnist.data.to_numpy()
labels = mnist.target.to_numpy()

In [6]:
train_data = data[:num_samples, :]
train_label = labels[:num_samples]
test_data = data[num_samples:, :]
test_label = labels[num_samples:]

In [7]:
enc = OneHotEncoder()
enc.fit(train_label[:, None])
train_encoded = enc.transform(train_label[:, None]).toarray()

In [8]:
train_reader = BatchReader(train_data, train_encoded)

In [9]:
enc = OneHotEncoder()
enc.fit(test_label[:, None])
test_encoded = enc.transform(test_label[:, None]).toarray()
test_reader = BatchReader(test_data, test_encoded)

In [10]:
dimensions = 784
classes = 10
hidden_layers = 3
hidden_layers_neurons = 400

In [11]:
input = C.input_variable(dimensions)
label = C.input_variable(classes)

In [12]:
def create_model(features):
    with C.layers.default_options(init=C.layers.glorot_uniform(), activation=C.ops.relu):
        hidden_out = features
        for _ in range(hidden_layers):
            hidden_out = C.layers.Dense(hidden_layers_neurons)(hidden_out)
        network_output = C.layers.Dense(classes, activation=None)(hidden_out)
        return network_output

In [13]:
model = create_model(input / 255.0)

In [14]:
loss = C.cross_entropy_with_softmax(model, label)
label_error = C.classification_error(model, label)

In [15]:
lrs = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
learner = C.sgd(model.parameters, lrs)
trainer = C.Trainer(model, (loss, label_error), [learner])

In [16]:
epochs = 10
num_iters = (num_samples * epochs) / batch_size
for i in range(int(num_iters)):
    batch_data, batch_label = train_reader.next_batch(batch_size=batch_size)
    arguments = {
        input: batch_data,
        label: batch_label
    }
    trainer.train_minibatch(arguments=arguments)
    if i % 1000 == 0:
        training_loss = trainer.previous_minibatch_loss_average
        evaluation_error = trainer.previous_minibatch_evaluation_average
        print("{0}: , Loss: {1:.3f}, Error: {2:.2f}%".format(i, training_loss, evaluation_error * 100))

C:\Users\yadar\Anaconda3\envs\handwritingrecognition\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input3") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
C:\Users\yadar\Anaconda3\envs\handwritingrecognition\lib\site-packages\cntk\core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input4") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


0: , Loss: 2.328, Error: 95.31%
1000: , Loss: 0.155, Error: 6.25%
2000: , Loss: 0.097, Error: 3.12%
3000: , Loss: 0.040, Error: 0.00%
4000: , Loss: 0.073, Error: 1.56%
5000: , Loss: 0.008, Error: 0.00%
6000: , Loss: 0.008, Error: 0.00%
7000: , Loss: 0.012, Error: 0.00%
8000: , Loss: 0.003, Error: 0.00%
9000: , Loss: 0.020, Error: 1.56%


In [17]:
predicted_label_probs = model.eval({input: test_data})
predictions = np.argmax(predicted_label_probs, axis=1)
actual = np.argmax(test_encoded, axis=1)
correct = np.sum(predictions == actual)
print(correct / len(actual))

0.9827


C:\Users\yadar\Anaconda3\envs\handwritingrecognition\lib\site-packages\cntk\core.py:82: RuntimeWarning: data is not C contiguous; rearrange your data/computation to avoid costly data conversions
  RuntimeWarning)


In [18]:
model.save("cntk.model")